# ER estimator 

We import all the necessary packages. We are going to work with the [fastai V1 library](http://www.fast.ai/2018/10/02/fastai-ai/) which sits on top of [Pytorch 1.0](https://hackernoon.com/pytorch-1-0-468332ba5163). The fastai library provides many useful functions that enable us to quickly and easily build neural networks and train our models.

In [3]:
from fastai.tabular import *
from fastai.metrics import error_rate

## Other data formats

In [4]:
 path = Path('../datasets')
(path/"").ls()

[PosixPath('../datasets/.~lock.20190919-to-20190921.csv#'),
 PosixPath('../datasets/20190912-to-20190918.csv'),
 PosixPath('../datasets/20190919-to-20190921.csv')]

In [6]:
df = pd.read_csv(path/'20190919-to-20190921.csv')
df = df.drop(columns="sid")
df = df.drop(columns="time")
df.head()

,g_event_id,r_dow,r_hod,a_er_cat,a_app_id,a_campaign_type_id,a_camp_id,a_group_id,r_source_id,c_id,...,r_platform,r_network_type,r_traffic_type,c_type,c_cta,c_text,c_size,s_gerry_type,s_caesar_id,response
0,-4,0,19,0,2214,2,6732,15102,69,6372771,...,ANDROID,cellular,app,HTML,XEM NGAY,ƯU ĐÃI ĐẾN 60%,3200480,NaN,NaN,0
1,-4,0,20,0,1087,2,25179,25740,253,6371282,...,ANDROID,wifi,app,IMAGE,NaN,NaN,3200480,NaN,NaN,0
2,-4,0,20,0,1061,2,25401,26160,69,6381865,...,ANDROID,cellular,app,HTML,Pesan Sekarang,"Saat waktu tak memberi \nbanyak pilihan, cek G...",4800320,0.0,NaN,0
3,-4,0,20,0,2214,2,6732,15102,259,6372757,...,ANDROID,wifi,app,HTML,XEM NGAY,ƯU ĐÃI ĐẾN 60%,3200480,NaN,NaN,0
4,-4,0,20,0,1088,2,25181,26511,74,6381396,...,IOS,wifi,app,IMAGE,NaN,NaN,3000250,NaN,NaN,0


In [23]:
df.columns

Index(['g_event_id', 'sid', 'time', 'r_dow', 'r_hod', 'a_er_cat', 'a_app_id',
       'a_campaign_type_id', 'a_camp_id', 'a_group_id', 'r_source_id', 'c_id',
       'r_site_id', 'r_site_iab_cats', 'r_country_id', 'r_region_code',
       'r_platform', 'r_network_type', 'r_traffic_type', 'c_type', 'c_cta',
       'c_text', 'c_size', 's_gerry_type', 's_caesar_id', 'response'],
      dtype='object')

We can define a bunch of Transforms that will be applied to our variables. Here we transform all categorical variables into categories. We also replace missing values for continuous variables by the median column value and normalize those.

In [25]:
procs = [FillMissing, Categorify, Normalize]

To split our data into training and validation sets, we use valid indexes

In [30]:
valid_idx = range(len(df)-100000, len(df))
valid_idx

range(860384, 960384)

let's split our variables into categorical and continuous variables (we can ignore the label variable at this stage). fastai will assume all variables that aren't labels or categorical are continuous, unless we explicitly pass a list to the cont_names parameter when constructing our DataBunch.

In [33]:
label = 'response'
cat_names = ['g_event_id', 'r_dow', 'r_hod', 'a_er_cat', 'a_app_id','a_campaign_type_id', 'a_camp_id', 'a_group_id', 'r_source_id', 'c_id','r_site_id', 'r_site_iab_cats', 'r_country_id', 'r_region_code','r_platform', 'r_network_type', 'r_traffic_type', 'c_type', 'c_cta','c_text', 'c_size', 's_gerry_type', 's_caesar_id', 'response']
cat_names

['g_event_id',
 'sid',
 'time',
 'r_dow',
 'r_hod',
 'a_er_cat',
 'a_app_id',
 'a_campaign_type_id',
 'a_camp_id',
 'a_group_id',
 'r_source_id',
 'c_id',
 'r_site_id',
 'r_site_iab_cats',
 'r_country_id',
 'r_region_code',
 'r_platform',
 'r_network_type',
 'r_traffic_type',
 'c_type',
 'c_cta',
 'c_text',
 'c_size',
 's_gerry_type',
 's_caesar_id',
 'response']